In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
import pandas as pd 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')


import textblob,string
from textblob import TextBlob

from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.preprocessing import text,sequence
from keras import layers,models,optimizers,regularizers
import matplotlib.pyplot as plt

from sklearn import model_selection,preprocessing,linear_model,naive_bayes,metrics
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn import decomposition,ensemble

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Pooja\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pooja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<a id = "1"></a><br>
## Load Libraries

In [11]:
#libraries
from keras.models import Sequential 
from keras.layers import Dense 
import matplotlib.pyplot as plt

#
from textblob import TextBlob
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
#
from sklearn import metrics
#
import textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
#
#stopwords
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#
from warnings import filterwarnings
filterwarnings('ignore')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
seed = 7 
np.random.seed(seed)
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [23]:
train=pd.read_csv('./Train.csv')
test=pd.read_csv('./Test.csv')
val=pd.read_csv('./Valid.csv')

In [14]:
train.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [7]:
train['text']

0        I grew up (b. 1965) watching and loving the Th...
1        When I put this movie in my DVD player, and sa...
2        Why do people who do not know what a particula...
3        Even though I have great interest in Biblical ...
4        Im a die hard Dads Army fan and nothing will e...
                               ...                        
39995    "Western Union" is something of a forgotten cl...
39996    This movie is an incredible piece of work. It ...
39997    My wife and I watched this movie because we pl...
39998    When I first watched Flatliners, I was amazed....
39999    Why would this film be so good, but only gross...
Name: text, Length: 40000, dtype: object

In [15]:
def transformation(df):
    df['text']=df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    df['text']=df['text'].str.replace('[^\w\s]','')
    df['text']=df['text'].str.replace('\d','')
    sw=stopwords.words('english')
    df['text']=df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    s=pd.Series(' '.join(df['text']).split()).value_counts()[-1000:]
    from textblob import Word
    df['text']=df['text'].apply(lambda x:' '.join(x for x in  x.split() if x not in s))

    df['text']=df['text'].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))
    return df

In [16]:
train=transformation(train)
train.head()

,text,label
0,grew (b. 1965) watching loving thunderbirds. m...,0
1,"put movie dvd player, sat coke chips, expectat...",0
2,people know particular time past like feel nee...,0
3,"even though great interest biblical movies, bo...",0
4,im die hard dad army fan nothing ever change t...,1


In [24]:
val= transformation(val)
val.head()

,text,label
0,14 year since sharon stone awarded viewer leg-...,0
1,someone needed make car payment... truly awful...,0
2,guideline state comment must contain minimum f...,0
3,movie muddled mish-mash clichés recent cinema....,0
4,stan laurel became smaller half all-time great...,0


In [25]:

test = transformation(test)
test.head()

,text,label
0,always wrote series complete stink-fest jim be...,0
1,1st watched 12/7/2002 - 3 10(dir-steve purcell...,0
2,movie poorly written directed fell asleep 30 m...,0
3,interesting thing miryang (secret sunshine) ac...,1
4,"first read ""berlin meer"" expect much. thought ...",0


In [26]:
x_train=train['text']
x_valid=val['text']
y_train=train['label']
y_val=val['label']


In [18]:
vectorizer=CountVectorizer() 
vectorizer.fit(x_train)

CountVectorizer()

In [27]:
x_train_count=vectorizer.transform(x_train)
x_valid_count=vectorizer.transform(x_valid)
x_test_count=vectorizer.transform(test['text'])

In [32]:
model=Sequential()
model.add(Dense(50,input_dim=x_train_count.shape[1],activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy']) 

history=model.fit(x_train_count,y_train.values.reshape(-1,1),validation_data=(x_valid_count,y_val),epochs=5,batch_size=128)

Epoch 1/5
313/313 [==============================] - 22s 69ms/step - loss: 0.3185 - accuracy: 0.8776
Epoch 2/5
313/313 [==============================] - 21s 67ms/step - loss: 0.1455 - accuracy: 0.9498
Epoch 3/5
313/313 [==============================] - 21s 67ms/step - loss: 0.0849 - accuracy: 0.9737
Epoch 4/5
313/313 [==============================] - 21s 67ms/step - loss: 0.0495 - accuracy: 0.9871
Epoch 5/5
313/313 [==============================] - 22s 69ms/step - loss: 0.0277 - accuracy: 0.9942


In [34]:
# loss,acc=model.evaluate(x_test_count,test['label'],verbose=1)
# print('Test Accuracy: %f' % (acc*100))

In [23]:
comments=pd.Series(test['text'])
comments=vectorizer.transform(comments)

<a id = "16"></a><br>
## Making Prediction

In [19]:
comment_1 = pd.Series("this film is very nice and good i like it")
comment_2 = pd.Series("no not good look at that shit very bad")

In [ ]:
comment_1

In [20]:
comment_1  = vectorizer.transform(comment_1)
comment_2 = vectorizer.transform(comment_2)

<a id = "17"></a><br>
## A Positive Comment

In [38]:
model.predict(comment_1)

1/1 [==============================] - 0s 24ms/step


array([[0.75256246]], dtype=float32)

<a id = "18"></a><br>
## A Negative Comment

In [39]:
model.predict(comment_2)

1/1 [==============================] - 0s 24ms/step


array([[0.21218699]], dtype=float32)

In [40]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

In [41]:
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [56]:
comment_3 = pd.Series("this does not look great")

In [57]:
comment_3= vectorizer.transform(comment_3)

In [58]:
loaded_model.predict(comment_3)

1/1 [==============================] - 0s 39ms/step


array([[0.55393434]], dtype=float32)

In [63]:
comment_4= pd.Series("this is bleh")
comment_4= vectorizer.transform(comment_4)
loaded_model.predict(comment_4)

1/1 [==============================] - 0s 26ms/step


array([[0.34826738]], dtype=float32)

In [54]:
comment_5= pd.Series("this sucks")
comment_5= vectorizer.transform(comment_5)
loaded_model.predict(comment_5)

1/1 [==============================] - 0s 163ms/step


array([[0.25327677]], dtype=float32)

In [ ]:
import streamlit as st
st.title('Sentiment Analysis Chatbot')
prompt = st.text_input('Enter a sentence')
if prompt:
    comment_n=pd.Series(prompt)
    comment_n=vectorizer.transform(comment_n)
    st.write(comment_n)
else:
    st.write('Please enter a sentence')